In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
observations = pd.read_csv('https://raw.githubusercontent.com/noo-rashbass/synthetic-data-service/master/Aisha/observations.csv')

Select some categorical columns in the data, carry out Chi-square test between every pair of them to see possible dependence relationships.

In [103]:
from scipy.stats import chi2_contingency
observ_columns = (observations.keys()).tolist()
categorical_index = ['Abdominal pain [HP_0002027]','Anorexia [SYMP_0000523]','Asexual Plasmodium parasites present, by microscopy [EUPATH_0000048]','Basis of complicated diagnosis [EUPATH_0000316]','Complicated malaria [EUPATH_0000040]','Cough [SYMP_0000614]','Diarrhea [DOID_13250]','Fatigue [SYMP_0019177]','Febrile [EUPATH_0000097]','Headache [HP_0002315]','ITN last night [EUPATH_0000216]','Jaundice [HP_0000952]','Joint pains [SYMP_0000064]','Malaria diagnosis [EUPATH_0000090]','Malaria diagnosis and parasite status [EUPATH_0000338]','Malaria treatment [EUPATH_0000740]','Muscle aches [EUPATH_0000252]','Plasmodium gametocytes present, by microscopy [EUPATH_0000207]','Seizures [SYMP_0000124]','Subjective fever [EUPATH_0000100]','Submicroscopic Plasmodium present, by LAMP [EUPATH_0000487]','Vomiting [HP_0002013]']
n = len(categorical_index)
ini = [ [ None for y in range( n ) ] 
             for x in range( n ) ]
ini_table = pd.DataFrame(ini,index = categorical_index,columns = categorical_index)
independence_list = []
highly_dependence_list = []
for i in range(n):
    for j in range(i+1,n):
        name1 = categorical_index[i]
        name2 = categorical_index[j]
        obs_1 = observations[name1].dropna()
        obs_2 = observations[name2].dropna()
        dataframe = pd.DataFrame({name1: obs_1, name2: obs_2})
        dataframe_stack = dataframe.groupby([name1,name2]).size()
        O_table = dataframe_stack.unstack().fillna(0)
        if (not O_table.empty):
            (statistic, p, dof, e) = chi2_contingency(O_table.values)
            ini_table.loc[name1,name2]=p
            ini_table.loc[name2,name1]=p
        else:
            ini_table.loc[name1,name2]='empty'
            ini_table.loc[name2,name1]='empty'
        if (0.7<p<1):
            independence_list.append((i,j,p))
        if (p==1):
            highly_dependence_list.append((i,j))
            
display(ini_table)
        

,Abdominal pain [HP_0002027],Anorexia [SYMP_0000523],"Asexual Plasmodium parasites present, by microscopy [EUPATH_0000048]",Basis of complicated diagnosis [EUPATH_0000316],Complicated malaria [EUPATH_0000040],Cough [SYMP_0000614],Diarrhea [DOID_13250],Fatigue [SYMP_0019177],Febrile [EUPATH_0000097],Headache [HP_0002315],...,Joint pains [SYMP_0000064],Malaria diagnosis [EUPATH_0000090],Malaria diagnosis and parasite status [EUPATH_0000338],Malaria treatment [EUPATH_0000740],Muscle aches [EUPATH_0000252],"Plasmodium gametocytes present, by microscopy [EUPATH_0000207]",Seizures [SYMP_0000124],Subjective fever [EUPATH_0000100],"Submicroscopic Plasmodium present, by LAMP [EUPATH_0000487]",Vomiting [HP_0002013]
Abdominal pain [HP_0002027],None,0,1.05375e-128,0.00306563,0.130297,4.42231e-252,0,0,0,0,...,0,9.51767e-219,0,2.83043e-214,0,0.0134985,1.02673e-27,0,2.59741e-16,4.62849e-229
Anorexia [SYMP_0000523],0,None,3.78461e-179,0.27976,0.0784821,4.22529e-127,6.07901e-311,0,0,0,...,0,0,0,0,0,0.0512477,2.98857e-13,0,0.19779,0
"Asexual Plasmodium parasites present, by microscopy [EUPATH_0000048]",1.05375e-128,3.78461e-179,None,1,1,1.64235e-26,0.433804,1.06763e-221,0,0,...,3.25771e-118,0,0,0,5.71897e-145,1.5378e-221,6.73973e-12,0,1,6.34532e-295
Basis of complicated diagnosis [EUPATH_0000316],0.00306563,0.27976,1,None,1,0.561941,0.485331,0.124489,1,0.00113622,...,0.0330453,1,1,1,0.107345,0.282222,0.00101053,1,empty,0.000156591
Complicated malaria [EUPATH_0000040],0.130297,0.0784821,1,1,None,0.0721839,0.128867,0.14463,1,0.00122307,...,0.746704,1,1,0,0.788142,0.610262,5.23268e-170,0.29204,empty,6.70807e-13
Cough [SYMP_0000614],4.42231e-252,4.22529e-127,1.64235e-26,0.561941,0.0721839,None,0.00054501,0,0,0,...,0,4.09431e-87,0,3.49413e-83,0,0.967585,9.4107e-07,0,8.34266e-05,2.18776e-53
Diarrhea [DOID_13250],0,6.07901e-311,0.433804,0.485331,0.128867,0.00054501,None,2.25868e-09,8.08069e-165,5.89062e-22,...,3.44282e-09,0.265196,0,0.572499,0.000111716,0.0599366,7.07745e-06,1.51433e-164,0.442137,0
Fatigue [SYMP_0019177],0,0,1.06763e-221,0.124489,0.14463,0,2.25868e-09,None,0,0,...,0,0,0,0,0,6.7794e-10,5.48167e-12,0,5.58786e-87,1.8221e-217
Febrile [EUPATH_0000097],0,0,0,1,1,0,8.08069e-165,0,None,0,...,0,0,0,0,0,0.0383068,1.38169e-20,0,1.45708e-18,0
Headache [HP_0002315],0,0,0,0.00113622,0.00122307,0,5.89062e-22,0,0,None,...,0,0,0,0,0,0.000351385,1.50221e-36,0,1.91846e-10,5.87477e-188


The table contains p values of the Chi-square test between two columns, the larger the p, the more confident we are to say that the two categories are independent.
The list 'independence_list' record (the index) of pairs of columns which have p > 0.7 i.e. not that dependent.
However, because of the way I record data in the code, p = 1 doesn't mean that we are 100% sure that the twocategories are independent, on the contrary, it means they are certainly related, please see the next section. The list 'highly_dependence_list' records such columns

In [104]:
def compare_two_categories(name1,name2):
    obs_1 = observations[name1].dropna()
    t1 = obs_1.groupby([obs_1]).count()
    display(t1)
    obs_2 = observations[name2].dropna()
    display(obs_2.groupby([obs_2]).count())
    dataframe = pd.DataFrame({name1: obs_1, name2: obs_2})
    dataframe_stack = dataframe.groupby([name1,name2]).size()
    O_table = dataframe_stack.unstack().fillna(0)
    display(O_table)
    
for (i,j) in highly_dependence_list:
    name1 = categorical_index[i]
    name2 = categorical_index[j]
    compare_two_categories(name1, name2)

Asexual Plasmodium parasites present, by microscopy [EUPATH_0000048]
Negative    33655
Positive     9141
Name: Asexual Plasmodium parasites present, by microscopy [EUPATH_0000048], dtype: int64

Basis of complicated diagnosis [EUPATH_0000316]
only danger signs                  25
only parasite density > 500,000    46
severe malaria                     14
Name: Basis of complicated diagnosis [EUPATH_0000316], dtype: int64

Basis of complicated diagnosis [EUPATH_0000316],only danger signs,"only parasite density > 500,000",severe malaria
"Asexual Plasmodium parasites present, by microscopy [EUPATH_0000048]",,,
Positive,25,46,14


Asexual Plasmodium parasites present, by microscopy [EUPATH_0000048]
Negative    33655
Positive     9141
Name: Asexual Plasmodium parasites present, by microscopy [EUPATH_0000048], dtype: int64

Complicated malaria [EUPATH_0000040]
No     5930
Yes      85
Name: Complicated malaria [EUPATH_0000040], dtype: int64

...

Submicroscopic Plasmodium present, by LAMP [EUPATH_0000487]
Negative     17860
Positive      4561
no result       10
Name: Submicroscopic Plasmodium present, by LAMP [EUPATH_0000487], dtype: int64

"Submicroscopic Plasmodium present, by LAMP [EUPATH_0000487]",Negative,Positive,no result
Malaria diagnosis [EUPATH_0000090],,,
No,17860,4561,10


Malaria treatment [EUPATH_0000740]
Artmether-lumefantrine for uncomplicated malaria                                         5884
No malaria medications given                                                            42706
Quinine for uncomplicated malaria in the 1st trimester of pregnancy                         9
Quinine for uncomplicated malaria within 14 days of a previous treatment for malaria       37
Quinine or Artesunate for complicated malaria                                              85
Name: Malaria treatment [EUPATH_0000740], dtype: int64

Submicroscopic Plasmodium present, by LAMP [EUPATH_0000487]
Negative     17860
Positive      4561
no result       10
Name: Submicroscopic Plasmodium present, by LAMP [EUPATH_0000487], dtype: int64

"Submicroscopic Plasmodium present, by LAMP [EUPATH_0000487]",Negative,Positive,no result
Malaria treatment [EUPATH_0000740],,,
No malaria medications given,17860,4561,10


There are some 'logic' relationships between the following columns:  
- Subjective fever [EUPATH_0000100] 
- Febrile [EUPATH_0000097]
- Asexual Plasmodium parasites present, by microscopy [EUPATH_0000048]
- Submicroscopic Plasmodium present, by LAMP [EUPATH_0000487]
- Malaria diagnosis [EUPATH_0000090]
- Complicated malaria [EUPATH_0000040] 
- Basis of complicated diagnosis [EUPATH_0000316]
- Malaria treatment [EUPATH_0000740]
- Malaria diagnosis and parasite status [EUPATH_0000338]

The relationships are drawn in the Analysis of highly_dependence_list in Pearson.pdf.
I call these relationships 'logic' because they are absolute - no outliers. For example, there is no data with 'No' in febrile or 'Negative' in Asexual microscopy but 'yes' in Malaria Diagnosis.

In [98]:
for (i,j,p) in independence_list:
    print(categorical_index[i],'&',categorical_index[j],'p =',p)

Anorexia [SYMP_0000523] & ITN last night [EUPATH_0000216] p = 0.970649694872472
Complicated malaria [EUPATH_0000040] & Joint pains [SYMP_0000064] p = 0.7467037437745767
Complicated malaria [EUPATH_0000040] & Muscle aches [EUPATH_0000252] p = 0.7881418058745915
Cough [SYMP_0000614] & Plasmodium gametocytes present, by microscopy [EUPATH_0000207] p = 0.967585260882485
ITN last night [EUPATH_0000216] & Jaundice [HP_0000952] p = 0.9812664093095449
ITN last night [EUPATH_0000216] & Malaria diagnosis [EUPATH_0000090] p = 0.938647337368375
ITN last night [EUPATH_0000216] & Seizures [SYMP_0000124] p = 0.9482145820602474
ITN last night [EUPATH_0000216] & Submicroscopic Plasmodium present, by LAMP [EUPATH_0000487] p = 0.8188093028796838
Jaundice [HP_0000952] & Plasmodium gametocytes present, by microscopy [EUPATH_0000207] p = 0.8665611153313111


The above is the list of pairs of columns which we can say independent with reasonable confidence. Worth to notice that 'ITN last night [EUPATH_0000216]' is independent to several columns.